In [1]:
import tensorflow as tf 
import keras 


In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np 

In [3]:
class SelfAttention (keras.layers.Layer) :
    def __init__ (self,d_k) :
        super (SelfAttention , self).__init__()
        self.__d_k= d_k 
    
    def build (self,inputs) :
        self.Wq = self.add_weight(shape=(inputs[-1],self.__d_k),initializer="random_normal",trainable=True)
        self.Wk = self.add_weight(shape=(inputs[-1],self.__d_k),initializer="random_normal",trainable=True)
        self.Wv = self.add_weight(shape=(inputs[-1],self.__d_k),initializer="random_normal",trainable=True)

    def call (self,x) :
        Q = tf.matmul(self.Wq,x)
        K = tf.matmul(self.Wk,x)
        V = tf.matmul(self.Wv,x)

        scores = tf.nn.softmax(tf.matmul(Q,tf.transpose(K,perm=[0,2,1])) / tf.math.sqrt(self.__d_k),axis=-1) 
        output = tf.matmul(scores,V)
        return output

In [4]:
class Text_Classifier (keras.Model) :
    def __init__ (self,vocab_size,num_class,embedding_size,d_k) :
        super (Text_Classifier,self).__init__()
        self.embedding = keras.layers.Embedding(input_dim=vocab_size,output_dim=embedding_size)
        self.Attention = SelfAttention(d_k)
        self.Long_sort_Term_Memories = keras.layers.LSTM(32,return_sequences=True)
        self.Dense = keras.layers.Dense(256,activation='relu')
        self.outputs = keras.layers.Dense(num_class,activation='softmax')
        self.Layer_Normal = keras.layers.LayerNormalization()
        self.GlobalPooling1D = keras.layers.GlobalAveragePooling1D()
    
    def call (self,x) :
        x = self.embedding(x)
        x = self.Long_sort_Term_Memories(x)
        x = self.Layer_Normal(x)
        x = self.GlobalPooling1D(x)
        x = self.Dense(x)
        return self.outputs(x)

In [24]:
vocab_size = 200
d_k = 32
num_class = 5
Embbedding_size = 64
models = Text_Classifier(vocab_size=vocab_size,num_class=num_class,embedding_size=Embbedding_size,d_k = d_k)

In [25]:
models.build(input_shape=(None,30))

c:\Users\ASUS\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\layer.py:391: UserWarning: `build()` was called on layer 'text__classifier_5', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


In [27]:
models.summary()

Model: "text__classifier_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ self_attention_5                │ ?                      │   0 (unbuilt) │
│ (SelfAttention)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_3           │ ?                      │   0 (unbuilt) │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_3      │ ?                      │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [10]:
dataset = [
    ("Produk ini sangat bagus dan berkualitas!", 1.0),
    ("Saya kecewa dengan layanan yang diberikan.", 0.0),
    ("Hari ini cuacanya cukup mendung.", 0.5),
    ("Pelayanan di restoran ini sangat ramah.", 1.0),
    ("Film ini sangat membosankan dan tidak menarik.", 0.0),
    ("Berita ini tidak terlalu menarik bagi saya.", 0.5),
    ("Saya sangat menikmati makan di sini!", 1.0),
    ("Laptop ini memiliki spesifikasi yang buruk.", 0.0),
    ("Langit terlihat cerah hari ini.", 0.5),
    ("Aplikasi ini sangat membantu dalam pekerjaan saya.", 1.0),
    ("Saya tidak suka dengan kualitas produk ini.", 0.0),
    ("Konser tadi malam sangat luar biasa!", 1.0),
    ("Pemain itu mencetak gol spektakuler.", 1.0),
    ("Saya tidak merekomendasikan restoran ini.", 0.0),
    ("Smartphone ini memiliki fitur yang sangat canggih.", 1.0),
    ("Saya merasa tidak puas dengan kecepatan internet.", 0.0),
    ("Pertandingan tadi malam berlangsung sangat seru!", 1.0),
    ("Acara televisi ini cukup menghibur.", 1.0),
    ("Ponsel ini sering mengalami lag.", 0.0),
    ("Saya suka desain mobil terbaru ini.", 1.0),
    ("Makanannya terlalu asin untuk selera saya.", 0.0),
    ("Layanan di hotel ini sangat memuaskan.", 1.0),
    ("Saya merasa harga tiketnya terlalu mahal.", 0.0),
    ("Pemandangan dari kamar hotel ini sangat indah.", 1.0),
    ("Buku ini memberikan banyak wawasan baru.", 1.0),
    ("Film ini tidak sesuai dengan ekspektasi saya.", 0.0),
    ("Saya merasa netral terhadap perubahan ini.", 0.5),
    ("Tempat wisata ini sangat direkomendasikan!", 1.0),
    ("Saya tidak akan membeli produk ini lagi.", 0.0),
    ("Toko ini memiliki banyak variasi produk.", 1.0),
    ("Sistem operasinya sering mengalami crash.", 0.0),
    ("Saya menikmati perjalanan dengan kereta cepat ini.", 1.0),
    ("Makanan di sini biasa saja, tidak terlalu enak.", 0.5),
    ("Game ini sangat adiktif dan menyenangkan!", 1.0),
    ("Saya kecewa dengan ending film ini.", 0.0),
    ("Cuaca hari ini cukup normal, tidak terlalu panas atau dingin.", 0.5),
    ("Restoran ini memiliki harga yang terjangkau.", 1.0),
    ("Saya tidak suka cara mereka menangani pelanggan.", 0.0),
    ("Produk ini sesuai dengan deskripsi dan kualitasnya bagus.", 1.0),
    ("Saya menunggu terlalu lama untuk mendapatkan pesanan saya.", 0.0),
    ("Musiknya sangat menenangkan.", 1.0),
    ("Saya berharap fitur ini bisa lebih baik lagi.", 0.5),
    ("Laptop ini cepat panas dan baterainya boros.", 0.0),
    ("Saya merasa sangat puas dengan pengalaman berbelanja di sini.", 1.0),
    ("Terlalu banyak iklan dalam aplikasi ini.", 0.0),
    ("Film ini memberikan pelajaran hidup yang berharga.", 1.0),
    ("Saya tidak mengerti jalan cerita film ini.", 0.0),
    ("Pelayanan di bank ini sangat cepat dan profesional.", 1.0),
    ("Saya tidak mendapatkan apa yang saya harapkan dari produk ini.", 0.0),
    ("Pemandangan di gunung ini luar biasa!", 1.0),
    ("Taman ini cukup nyaman untuk bersantai.", 0.5),
    ("Saya merasa aplikasi ini terlalu sulit digunakan.", 0.0),
    ("Saya menikmati waktu saya di taman hiburan ini.", 1.0),
    ("Sinyal internet di daerah ini sangat buruk.", 0.0),
    ("Kopi ini memiliki rasa yang unik dan enak.", 1.0),
    ("Saya berharap pengiriman bisa lebih cepat.", 0.5),
    ("Saya tidak menyukai interaksi antar karakter dalam drama ini.", 0.0),
    ("Perjalanan dengan kapal pesiar ini sangat menyenangkan!", 1.0),
    ("Saya mengalami masalah saat mencoba menghubungkan perangkat ini.", 0.0),
    ("Saya merasa tidak ada yang spesial dengan produk ini.", 0.5),
]


In [46]:
x_train = list()
y_train = list() 
for i in range(len(dataset)) :
    x_train.append(dataset[i][0])
    y_train.append(dataset[i][1])

In [20]:
tokenizers = Tokenizer()

In [31]:
tokenizers.fit_on_texts(x_train)

In [47]:
x_train = tokenizers.texts_to_sequences(x_train)
x_train = pad_sequences(x_train,maxlen=10,padding='post')

In [49]:
y_train = np.array(y_train).reshape(-1,1)

In [51]:
vocab = len(tokenizers.index_word) + 1 

In [ ]:
inputs_models = keras.Input(shape=(1,10))
embedding = keras.layers.Embedding(input_dim=vocab,output_dim=32)(inputs_models)
query = keras.layers.Dense(32)(embedding)
keys = keras.layers.Dense(32)(embedding)
values = keras.layer
Attention_m = keras.layers.Attention()

'kontol'

In [73]:
import tensorflow as tf
from tensorflow import keras

# Input Shape (batch_size, 10)
inputs_models = keras.Input(shape=(10,))

# Embedding Layer
embedding_models = keras.layers.Embedding(input_dim=10000, output_dim=32)(inputs_models)

# Query, Key, Value untuk Self-Attention
query = keras.layers.Dense(32)(embedding_models)
keys = keras.layers.Dense(32)(embedding_models)
values = keras.layers.Dense(32)(embedding_models)

# Self-Attention (Output 3D: (batch_size, seq_len, features))
Attention_models = keras.layers.Attention()([query, keys, values])

# Gunakan GlobalAveragePooling1D karena output Attention masih 3D
Pooling_models = keras.layers.GlobalAveragePooling1D()(Attention_models)

# LSTM Layer (Hapus GlobalPooling di sini karena LSTM butuh 3D input)
Lstm_models = keras.layers.LSTM(16, return_sequences=True)(embedding_models)

# Layer Normalization
Layer_normal = keras.layers.LayerNormalization()(Lstm_models)

# Fully Connected Layers
Dense_models = keras.layers.Dense(32, activation='relu')(Layer_normal)
Dense2_models = keras.layers.Dense(16, activation='relu')(Dense_models)

# Output Layer (Sigmoid untuk Binary Classification)
outputs = keras.layers.Dense(1, activation='sigmoid')(Dense2_models)

# Compile Model
model = keras.Model(inputs=inputs_models, outputs=outputs)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Summary Model
model.summary()



Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_21 (InputLayer)     │ (None, 10)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_21 (Embedding)        │ (None, 10, 32)         │       320,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (None, 10, 16)         │         3,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_2           │ (None, 10, 16)         │            32 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_60 (Dense)                │ (None, 10, 32)         │           544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_61 (Dense)                │ (None, 10, 16)         │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_62 (Dense)                │ (None, 10, 1)          │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 324,257 (1.24 MB)

 Trainable params: 324,257 (1.24 MB)

 Non-trainable params: 0 (0.00 B)